# TV Script Generation
So who needs a simpsons script when they have been going on forever when we all want new Firefly episodes...

Ok, so that may never happen however but maybe this script generator can throw out a couple of good lines...

## Get the Data
I tracked down the scripts and poorly formatted them to make this work.

In [1]:
import helper

data_dir = './data/ff/script.txt'
text = helper.load_data(data_dir)

## Explore the Data

In [2]:
view_sentence_range = (0, 50)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 15387
Number of scenes: 586
Average number of sentences in each scene: 11.943686006825939
Number of lines: 7585
Average number of words in each line: 12.31641397495056

The sentences 0 to 50:
INT. BAR
Belly dancer and men playing cards, middle eastern music playing. Pan up MAL in brown coat playing chinese checkers. He's sitting with an ZOE and JAYNE
JAYNE: (to MAL) Your move.
MAL moves pieces.
ZOE: (to Mal) That's a bold move. 
MAL: I live on the edge. 
Mal takes a swallow of his drink. Zoe moves her pieces. 
JAYNE: (to MAL) Nice work, dumbass. 
MAL: I'd given some thought to movin' off the edge -- not an ideal location -- thinkin' a place in the middle.
MAN: (O.S.) Toast! Toast! 
the noise continues
MAN: QUIET! SHUT UP! 
(Camera moves to a MAN, standing at the bar. He's large, bald, and drunk -- and his name is LUND.)
LUND: I'm, uh, I got words... I'm sayin' this is an a'spicious day. 
(Camera moves to MAL, looking uncomfortable and s

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """

    'Set removes duplicates...'
    
    'need id int, could use counter and itterate through however if i ennumerate the set that should do'
    'first part of enumerate is the index which i can use for the id'
    'i can split the enumeration using list comprehension' 
           
    vocab_to_int = {the_word:the_index for the_index, the_word in enumerate(set(text))}
    int_to_vocab = {the_other_index:the_other_word for the_other_index, the_other_word in enumerate(set(text))}
    
    return (vocab_to_int, int_to_vocab)


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    
    'key:value'
    'create new dictionary'
    punctuation_dictionary  = {".":"||prd||", ",":"||cma||", "\"":"||qot||", ";":"||smi||", "!":"||exc||", "?":"||qtm||", "(":"||lpr||", ")":"||rpr||", "--":"||dsh||", "\n":"||rtn||"}
    
    return punctuation_dictionary


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
import helper
import numpy as np

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.10.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    Input = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='target')
    LearningRate = tf.placeholder(tf.float32, name='learningrate')
    return (Input, Targets, LearningRate)


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
# Number of Epochs
num_epochs = 10
# Number of Epochs
num_epochs_f = 1000
# Number of Epochs
num_laz = 1
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.0001
# Show stats for every n number of batches
show_every_n_batches = 1287

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

In [10]:
def get_init_cell(batch_size, rnn_size, d_out=1, n_of_l=num_laz):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    Cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, d_out) for _ in range(n_of_l)])
    InitialState = Cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(InitialState, 'initial_state')
    return (Cell, InitialState)



### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """    

    embed = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    return tf.nn.embedding_lookup(embed, input_data)



### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    Outputs, FinalState = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(FinalState, name='final_state')
    
    return (Outputs, FinalState)



### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """

    emb_layer = get_embed(input_data, vocab_size, embed_dim)
    outputs, FinalState = build_rnn(cell, emb_layer)
    Logits = tf.layers.dense(outputs, vocab_size, activation=None, use_bias=True)
    return (Logits, FinalState)


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

In [14]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
   
    n_batches=len(int_text)//(batch_size * seq_length)
    
    nx_arr = np.array(int_text[:n_batches * (batch_size  * seq_length)])            
    
    x_text = nx_arr.reshape(-1, seq_length)
    y_text = np.roll(nx_arr, -1).reshape(-1, seq_length)
    
    opt = np.zeros([n_batches, 2, batch_size, seq_length], dtype=np.int32)
    for idx in range(0, x_text.shape[0]):
        m = idx % n_batches
        n = idx // n_batches
        opt[m,0,n,:] = x_text[idx,:]
        opt[m,1,n,:] = y_text[idx,:]
    return opt


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        
        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/40   train_loss = 9.031
Model Trained and Saved


In [17]:
from tensorflow.contrib import seq2seq


r_check = False

# Number of Epochs
b_num_epochs = 0
# Number of Epochs
b_num_laz = 1
# Batch Size
b_batch_size = 0
# RNN Size
b_rnn_size = 0
# Embedding Dimension Size
b_embed_dim = 0
# Sequence Length
b_seq_length = 0
# Learning Rate
b_learning_rate = 1
# Show stats for every n number of batches
b_show_every_n_batches = 0
# Show stats for every n number of batches
b_t_loss = 50

# Number of Epochs
num_epochs = 1000
# Number of Epochs
num_epochs_f = 5000
b_show_every_n_epochs = 200


if r_check == True:
    from tensorflow.contrib import seq2seq
    #for batch_size_ in [128, 256]:
    #    for rnn_size_ in [128, 256, 512]:
    #        for num_laz_ in [1, 2, 3]:
    #            for learning_rate_ in [1, 0.1, 0.01, 0.001, 0.0001]:
    for batch_size_ in [256]:
        for rnn_size_ in [512]:
            for num_laz_ in [1]:
                for learning_rate_ in [0.0001]:
                    batch_size=batch_size_
                    rnn_size=rnn_size_
                    num_laz=num_laz_
                    learning_rate=learning_rate_
                    train_graph = tf.Graph()
                    with train_graph.as_default():
                        b_show_every_n_epochs
                        vocab_size = len(int_to_vocab)
                        input_text, targets, lr = get_inputs()
                        input_data_shape = tf.shape(input_text)
                        cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
                        logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

                        # Probabilities for generating words
                        probs = tf.nn.softmax(logits, name='probs')

                        # Loss function
                        cost = seq2seq.sequence_loss(
                            logits,
                            targets,
                            tf.ones([input_data_shape[0], input_data_shape[1]]))

                        # Optimizer
                        optimizer = tf.train.AdamOptimizer(lr)

                        # Gradient Clipping
                        gradients = optimizer.compute_gradients(cost)
                        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
                        train_op = optimizer.apply_gradients(capped_gradients)

                    batches = get_batches(int_text, batch_size, seq_length)

                    with tf.Session(graph=train_graph) as sess:
                        sess.run(tf.global_variables_initializer())
                        print('Batch Size {}, RNN Size {}, Hidden Layers {}, Learning rate {}'.format(
                                        batch_size,
                                        rnn_size,
                                        num_laz,
                                        learning_rate))
                        for epoch_i in range(num_epochs):
                            state = sess.run(initial_state, {input_text: batches[0][0]})


                            for batch_i, (x, y) in enumerate(batches):
                                feed = {
                                    input_text: x,
                                    targets: y,
                                    initial_state: state,
                                    lr: learning_rate}
                                train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
                            if (epoch_i + 1) % b_show_every_n_epochs == 0:
                                print(' - Epoch {:>3} Training Loss: {:.3f}'.format(
                                    epoch_i + 1,                                        
                                    train_loss))
                                # Show every <show_every_n_batches> batches

                        #print('Batch Size {}, RNN Size {}, Hidden Layers {}, Learning rate {}, train_loss = {}'.format(
                        #                batch_size,
                        #                rnn_size,
                        #                num_laz,
                        #                learning_rate,
                        #                train_loss))
                        if train_loss < b_t_loss:
                            b_batch_size = batch_size
                            b_rnn_size = rnn_size
                            b_num_laz = num_laz
                            b_learning_rate = learning_rate
                            b_t_loss = train_loss


    print('Best: Batch Size {}, RNN Size {}, Hidden Layers {}, Learning rate {}'.format(
                        b_batch_size,
                        b_rnn_size,
                        b_num_laz,
                        b_learning_rate))
    batch_size=b_batch_size
    rnn_size=b_rnn_size
    num_laz=b_num_laz
    learning_rate=b_learning_rate

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)    
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs_f):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        
        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

        # Show every <show_every_n_batches> batches
        if (epoch_i ) % 100 == 0:
            print('Epoch {:>3} train_loss = {:.3f}'.format(
                epoch_i,
                train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 train_loss = 8.210
Epoch 100 train_loss = 4.179
Epoch 200 train_loss = 3.292
Epoch 300 train_loss = 2.575
Epoch 400 train_loss = 1.974
Epoch 500 train_loss = 1.502
Epoch 600 train_loss = 1.138
Epoch 700 train_loss = 0.896
Epoch 800 train_loss = 0.685
Epoch 900 train_loss = 0.515
Epoch 1000 train_loss = 0.429
Epoch 1100 train_loss = 0.384
Epoch 1200 train_loss = 0.349
Epoch 1300 train_loss = 0.333
Epoch 1400 train_loss = 0.321
Epoch 1500 train_loss = 0.316
Epoch 1600 train_loss = 0.310
Epoch 1700 train_loss = 0.308
Epoch 1800 train_loss = 0.305
Epoch 1900 train_loss = 0.304
Epoch 2000 train_loss = 0.303
Epoch 2100 train_loss = 0.302
Epoch 2200 train_loss = 0.301
Epoch 2300 train_loss = 0.301
Epoch 2400 train_loss = 0.300
Epoch 2500 train_loss = 0.300
Epoch 2600 train_loss = 0.300
Epoch 2700 train_loss = 0.299
Epoch 2800 train_loss = 0.299
Epoch 2900 train_loss = 0.299
Epoch 3000 train_loss = 0.299
Epoch 3100 train_loss = 0.298
Epoch 3200 train_loss = 0.298
Epoch 3300 train_los

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [18]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [19]:
import tensorflow as tf
import numpy as np
import helper


_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [20]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """

    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [21]:
def pick_word(probabilities, int_to_vocab, top_n=5):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """

    
    p = np.squeeze(probabilities)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    #print(p)
    c = np.random.choice(list(int_to_vocab.values()), 1, p=p)[0]
    return c
    

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [22]:
gen_length = 10000
prime_word = 'mal'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        #print(probabilities)
        #pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)
        pred_word = pick_word(probabilities[: , dyn_seq_length-1], int_to_vocab)                

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
mal:(wheezes) how-- you-- do-- in'?

ext. space - continuous
mal, zoe and simon and zoe are seated in the train with wash.
zoe: something. you're not going to the doctor and yours?
zoe and mal nods to her feet. a beat) we'll start to--
nandi: you know.
wash nods, begins a gun toward mal's head, and walks off...
mal:(you're back to tell the ship... just my folks asking you and the rest of the thought. i don't want to do for?
it's not to get the damn so..
wash and mal nods, we see a bit more.
a beat, then turns off.) can be you keep the engine thought you, uh?
mal:(quietly takes a corridor) no!, i just don't look. the captain who's that out.

ext. bordello, we're just heavy the cargo hold on this piece, you and business under in more.
mal: you got a point of mine, and they are bound, and what's off her.
simon: you really think of a funny.?

int. new alliance ship - continuous
zoe. mal is walking. they exchange a stretcher where wash won't 